In [1]:
%matplotlib inline
import torch
import random
import pandas as pd
import numpy as np
import pandas_datareader.data as web
from matplotlib import pyplot
import seaborn as sns
import os

#Plotting 
from pandas.plotting import scatter_matrix

#Libraries for Statistical Models
import statsmodels.api as sm

#logging
from MyPyUtil.logconf import logging
log = logging.getLogger(__name__)
# log.setLevel(logging.ERROR)
log.setLevel(logging.INFO)
# log.setLevel(logging.WARN)
# log.setLevel(logging.DEBUG)


#Diable the warnings
import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None
pd.options.display.expand_frame_repr = False
pd.options.display.float_format = '{:.3f}'.format

torch.seed = 42
random.seed(42)
np.random.seed(42)

%run 'nb_utils.ipynb'
task_name = get_filename_of_ipynb()
print(task_name)
data_dir = f'{os.getcwd()}/data/'
log_dir_base = f'{os.getcwd()}/runs/{task_name}'
log_dir = log_dir_base
print(f'{data_dir}\n{log_dir}')


StockPCTLabelPredictLSTMV2
/mnt/AIWorkSpace/work/fin-ml/data/
/mnt/AIWorkSpace/work/fin-ml/runs/StockPCTLabelPredictLSTMV2


In [2]:
# hyperparameters turning
from ray import tune, train, ray
from ray.tune.schedulers import ASHAScheduler

ray.init(log_to_driver=False)

2024-02-08 06:02:16,799	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.8.18
Ray version:,2.9.1
Dashboard:,http://127.0.0.1:8265


In [21]:
from datetime import datetime
import yfinance as yfin
from MyPyUtil.util import is_contained

# Loading the data
stk_symbols = [
    "AAPL",
    "MSFT",
    "AMZN",
    "NVDA",
    "GOOGL",
    "GOOG",
    "META",
    "TSLA",
    "UNH",
    "LLY",
    "JPM",
    "XOM",
    "JNJ",
    "V",
    "PG",
    "AVGO",
    "MA",
    "HD",
    "CVX",
    "MRK",
    "ABBV",
    "PEP",
    "COST",
    "ADBE",
    "KO",
    "CSCO",
    "WMT",
    "TMO",
    "MCD",
    "PFE",
    "CRM",
    "BAC",
    "ACN",
    "CMCSA",
    "LIN",
    "NFLX",
    "ABT",
    "ORCL",
    "DHR",
    "AMD",
    "WFC",
    "DIS",
    "TXN",
    "PM",
    "VZ",
    "INTU",
    "COP",
    "CAT",
    "AMGN",
    "NEE",
    "INTC",
    "UNP",
    "LOW",
    "IBM",
    "BMY",
    "SPGI",
    "RTX",
    "HON",
    "BA",
    "UPS",
    "GE",
    "QCOM",
    "AMAT",
    "NKE",
    "PLD",
    "NOW",
    "BKNG",
    "SBUX",
    "MS",
    "ELV",
    "MDT",
    "GS",
    "DE",
    "ADP",
    "LMT",
    "TJX",
    "T",
    "BLK",
    "ISRG",
    "MDLZ",
    "GILD",
    "MMC",
    "AXP",
    "SYK",
    "REGN",
    "VRTX",
    "ETN",
    "LRCX",
    "ADI",
    "SCHW",
    "CVS",
    "ZTS",
    "CI",
    "CB",
    "AMT",
    "SLB",
    "C",
    "BDX",
    "MO",
    "PGR",
    "TMUS",
    "FI",
    "SO",
    "EOG",
    "BSX",
    "CME",
    "EQIX",
    "MU",
    "DUK",
    "PANW",
    "PYPL",
    "AON",
    "SNPS",
    "ITW",
    "KLAC",
    "LULU",
    "ICE",
    "APD",
    "SHW",
    "CDNS",
    "CSX",
    "NOC",
    "CL",
    "MPC",
    "HUM",
    "FDX",
    "WM",
    "MCK",
    "TGT",
    "ORLY",
    "HCA",
    "FCX",
    "EMR",
    "PXD",
    "MMM",
    "MCO",
    "ROP",
    "CMG",
    "PSX",
    "MAR",
    "PH",
    "APH",
    "GD",
    "USB",
    "NXPI",
    "AJG",
    "NSC",
    "PNC",
    "VLO",
    "GBP",
    "F",
    "MSI",
    "GM",
    "TT",
    "EW",
    "CARR",
    "AZO",
    "ADSK",
    "TDG",
    "ANET",
    "SRE",
    "ECL",
    "OXY",
    "PCAR",
    "ADM",
    "MNST",
    "KMB",
    "PSA",
    "CCI",
    "CHTR",
    "MCHP",
    "MSCI",
    "CTAS",
    "WMB",
    "AIG",
    "STZ",
    "HES",
    "NUE",
    "ROST",
    "AFL",
    "AEP",
    "IDXX",
    "D",
    "TEL",
    "JCI",
    "MET",
    "GIS",
    "IQV",
    "EXC",
    "WELL",
    "DXCM",
    "HLT",
    "ON",
    "COF",
    "PAYX",
    "TFC",
    "USD",
    "BIIB",
    "O",
    "FTNT",
    "DOW",
    "TRV",
    "DLR",
    "MRNA",
    "CPRT",
    "ODFL",
    "DHI",
    "YUM",
    "SPG",
    "CTSH",
    "AME",
    "BKR",
    "SYY",
    "A",
    "CTVA",
    "CNC",
    "EL",
    "AMP",
    # "CEG",  # PCT <= -0.05,  size = 0
    "HAL",
    # "OTIS",  # PCT <= -0.05,  size = 0
    "ROK",
    "PRU",
    "DD",
    "KMI",
    "VRSK",
    "LHX",
    "DG",
    "FIS",
    "CMI",
    "CSGP",
    "FAST",
    "PPG",
    "GPN",
    "GWW",
    "HSY",
    "BK",
    "XEL",
    "DVN",
    "EA",
    "NEM",
    "ED",
    "URI",
    "VICI",
    "PEG",
    "KR",
    "RSG",
    "LEN",
    "PWR",
    "WST",
    "COR",
    "OKE",
    "VMC",
    "KDP",
    "WBD",
    "ACGL",
    "ALL",
    "IR",
    "CDW",
    "FANG",
    "MLM",
    "PCG",
    "DAL",
    "EXR",
    "FTV",
    "AWK",
    "IT",
    "KHC",
    # "GEHC",  # PCT <= -0.05,  size = 0
    "WEC",
    "HPQ",
    "EIX",
    "CBRE",
    "APTV",
    "ANSS",
    "MTD",
    "DLTR",
    "AVB",
    "ILMN",
    "ALGN",
    "LYB",
    "TROW",
    "GLW",
    "EFX",
    "WY",
    "ZBH",
    "XYL",
    "SBAC",
    "RMD",
    "TSCO",
    "EBAY",
    "KEYS",
    "CHD",
    "STT",
    "DFS",
    "HIG",
    "ALB",
    "STE",
    "ES",
    "TTWO",
    "MPWR",
    "CAH",
    "EQR",
    "RCL",
    "WTW",
    "HPE",
    "DTE",
    "GPC",
    "BR",
    "ULTA",
    "FICO",
    "CTRA",
    "BAX",
    "AEE",
    "MTB",
    "MKC",
    "ETR",
    "WAB",
    "DOV",
    "FE",
    "RJF",
    "INVH",
    "FLT",
    "CLX",
    "TDY",
    "TRGP",
    "DRI",
    "LH",
    "HOLX",
    "VRSN",
    "MOH",
    "LUV",
    "PPL",
    "ARE",
    "NVR",
    "COO",
    "WBA",
    "PHM",
    "NDAQ",
    "HWM",
    "RF",
    "CNP",
    "IRM",
    "LVS",
    "FITB",
    "EXPD",
    "VTR",
    "FSLR",
    "PFG",
    "BRO",
    "J",
    "IEX",
    "BG",
    "ATO",
    "FDS",
    "ENPH",
    "MAA",
    "CMS",
    "IFF",
    "BALL",
    "SWKS",
    "CINF",
    "NTAP",
    "STLD",
    "UAL",
    "WAT",
    "OMC",
    "TER",
    "CCL",
    "JBHT",
    "MRO",
    "TYL",
    "HBAN",
    "K",
    "GRMN",
    "CBOE",
    "NTRS",
    "TSN",
    "AKAM",
    "EG",
    "ESS",
    "EQT",
    "TXT",
    "EXPE",
    "SJM",
    "PTC",
    "DGX",
    "AVY",
    "RVTY",
    "BBY",
    "CF",
    "CAG",
    "EPAM",
    "AMCR",
    "LW",
    "PAYC",
    "SNA",
    "AXON",
    "POOL",
    "SYF",
    "SWK",
    "ZBRA",
    "DPZ",
    "PKG",
    "CFG",
    "LDOS",
    "VTRS",
    "PODD",
    "LKQ",
    "MOS",
    "APA",
    "EVRG",
    "TRMB",
    "MGM",
    "NDSN",
    "WDC",
    "MAS",
    "LNT",
    "IPG",
    "MTCH",
    "STX",
    "KMX",
    "TECH",
    "WRB",
    "LYV",
    "IP",
    "UDR",
    "AES",
    "CE",
    "INCY",
    "L",
    "TAP",
    "GEN",
    "CPT",
    "KIM",
    "JKHY",
    "HRL",
    "HST",
    "FMC",
    "CZR",
    "PEAK",
    "CDAY",
    "PNR",
    "NI",
    "CHRW",
    "HSIC",
    "CRL",
    "REG",
    "QRVO",
    "TFX",
    "KEY",
    "GL",
    "EMN",
    "WYNN",
    "ALLE",
    "AAL",
    "FFIV",
    "BWA",
    "BXP",
    "MKTX",
    "ROL",
    "JNPR",
    "PNW",
    "ETSY",
    "BLDR",
    "FOXA",
    "AOS",
    "HAS",
    "HII",
    "NRG",
    "CPB",
    "UHS",
    "BIO",
    "WRK",
    "RHI",
    "CTLT",
    "XRAY",
    "BBWI",
    "NWSA",
    "TPR",
    "PARA",
    "WHR",
    "BEN",
    "AIZ",
    "NCLH",
    "GNRC",
    "FRT",
    "IVZ",
    "VFC",
    "CMA",
    "DVA",
    "JBL",
    "HUBB",
    "ZION",
    "UBER",
    "MHK",
    "RL",
    "FOX",
    "BX",
    "ABNB",
    "NWS",
]
# stk_symbols = [
#     "AAPL",
#     "MSFT",
#     "AMZN",
#     "NVDA",
#     "GOOGL",
#     "TSLA",
#     "META",
#     "GOOG",
#     "ADBE",
#     "NFLX",
#     "CSCO",
#     "INTC",
#     "INTU",
#     "CMCSA",
#     "TXN",
#     "AMAT",
#     "ADSK",
#     "AMD",
#     "QCOM",
#     "MU",
# ]

# stk_symbols = [
#     "AAPL",
#     "MSFT",
#     "AMZN",
#     "NVDA",
#     "GOOGL",
#     "TSLA",
#     "META",
#     "GOOG",
# ]

empty_vol_threshold = 5
start = datetime(2014, 1, 1)
end = datetime(2023, 12, 31)

ticks_data = []
for symbol in stk_symbols:
    stk_file = f"{data_dir}{symbol}.csv"
    bLoad = False
    if os.path.isfile(stk_file):
        try:
            _stk_data = pd.read_csv(stk_file).set_index("Date")
            bLoad = True
            print(f"read {stk_file} completely!")
        except:
            None
    if bLoad == False:
        # _stk_data = web.get_data_yahoo(stk_tickers, start, end)
        _stk_data = yfin.download([symbol], start, end).dropna()
        _stk_data.to_csv(stk_file)
        print(f"download {symbol} from yfin and write to {stk_file} completely!")

    statistics = _stk_data.describe()
    if is_contained(statistics, 0):
        if is_contained(
            statistics.loc[:, ["Open", "High", "Low", "Close", "Adj Close"]], 0
        ) or is_contained(statistics.loc["std"], 0):
            print(f"{symbol}: contains numerical errors. Ignore it.")
            continue
        else:
            empty_vol_index = _stk_data[_stk_data["Volume"] == 0].index
            if len(empty_vol_index) > empty_vol_threshold:
                print(
                    f"The total volume with a value of zero ({len(empty_vol_index)}) is greater than the threshold({empty_vol_threshold}). Ignore it."
                )
                continue
            print(
                f"A total of {len(empty_vol_index)} volume values ​​are zero. Delete these data."
            )

            cleaned_data = _stk_data.drop(empty_vol_index)
            print(
                f"The cleaned data size is {len(cleaned_data)}. The original data size is {len(_stk_data)}."
            )
            if len(cleaned_data) == 0:
                continue
            _stk_data = cleaned_data

    ticks_data.append(_stk_data)
    print(f"{symbol}, size:{len(_stk_data)}")

read /mnt/AIWorkSpace/work/fin-ml/data/AAPL.csv completely!
AAPL, size:2516
read /mnt/AIWorkSpace/work/fin-ml/data/MSFT.csv completely!
MSFT, size:2516
read /mnt/AIWorkSpace/work/fin-ml/data/AMZN.csv completely!
AMZN, size:2516
read /mnt/AIWorkSpace/work/fin-ml/data/NVDA.csv completely!
NVDA, size:2516
read /mnt/AIWorkSpace/work/fin-ml/data/GOOGL.csv completely!
GOOGL, size:2516
read /mnt/AIWorkSpace/work/fin-ml/data/GOOG.csv completely!
GOOG, size:2516
read /mnt/AIWorkSpace/work/fin-ml/data/META.csv completely!
META, size:2516
read /mnt/AIWorkSpace/work/fin-ml/data/TSLA.csv completely!
TSLA, size:2516
read /mnt/AIWorkSpace/work/fin-ml/data/UNH.csv completely!
UNH, size:2516
read /mnt/AIWorkSpace/work/fin-ml/data/LLY.csv completely!
LLY, size:2516
read /mnt/AIWorkSpace/work/fin-ml/data/JPM.csv completely!
JPM, size:2516
read /mnt/AIWorkSpace/work/fin-ml/data/XOM.csv completely!
XOM, size:2516
read /mnt/AIWorkSpace/work/fin-ml/data/JNJ.csv completely!
JNJ, size:2516
read /mnt/AIWorkSpac

In [18]:
stk_data = ticks_data[39]
print(len(stk_data))
describe = stk_data.describe()
print(describe)
aa = (describe == 0).any(axis=1)
print(len(describe[aa]))
print(is_contained(describe, 0))
print((stk_data["Volume"] == 0).sum())
if (stk_data["Volume"] == 0).sum() > 0:
    aa = stk_data.drop(stk_data[stk_data["Volume"] == 0].index)
    print(aa.describe())
# print(pd.isnu describe[describe > 0])
# for i, stk_data in enumerate(ticks_data):
#     if (stk_data.describe() == 0).sum() > 0:
#         print(f"{i}, {stk_symbols[i]}\n,{stk_data.describe()}")

2516
          Open     High      Low    Close  Adj Close        Volume
count 2516.000 2516.000 2516.000 2516.000   2516.000      2516.000
mean    43.107   43.990   42.206   43.115     43.115  54964631.876
std     42.265   43.114   41.379   42.262     42.262  37508800.880
min      1.620    1.690    1.610    1.620      1.620         0.000
25%      5.097    5.190    4.990    5.100      5.100  29457475.000
50%     23.645   24.210   23.215   23.680     23.680  48602500.000
75%     82.067   83.462   80.595   82.003     82.003  73106175.000
max    163.280  164.460  156.100  161.910    161.910 325058400.000
1
True
1
          Open     High      Low    Close  Adj Close        Volume
count 2515.000 2515.000 2515.000 2515.000   2515.000      2515.000
mean    43.123   44.006   42.221   43.131     43.131  54986486.600
std     42.266   43.115   41.380   42.263     42.263  37500234.414
min      1.620    1.690    1.610    1.620      1.620   2606600.000
25%      5.115    5.190    5.000    5.110      5

In [22]:
import torch

device_name = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
device = torch.device(device_name)
return_period = 5
seq_len = 3
validation_size = 0.2
epoch_num = 100
batch_size = 32
num_workers = 0
pin_memory = False
shuffle = False
print(f"device_name:{device}")

device_name:cuda


In [23]:
pct_threshold = 0.05
class_percentage_threshold = 0.01  # percentage threshold for class size
classificationThreshold = 0.5


# # number of classes = 3
# # 0: PCT <= -0.05
# # 1: 0.05 < PCT < -0.05
# # 2: PCT >= 0.05
# num_classes = 3


# def gen_pct_label(stk_data, _return_period):
#     max_price_period = (
#         stk_data["Adj Close"].rolling(_return_period).max().shift(-_return_period)
#     )
#     max_pct_period = (max_price_period - stk_data["Adj Close"]) / stk_data["Adj Close"]
#     pct_label = max_pct_period.apply(
#         lambda x: 2 if x >= pct_threshold else 0 if x <= -pct_threshold else 1
#     ).astype("int8")
#     pct_label.name = "label"
#     return pct_label


# number of classes = 2
# 0: PCT < 0.05
# 1: PCT >= -0.05
num_classes = 2


def gen_pct_label(stk_data, _return_period):
    max_price_period = (
        stk_data["Adj Close"].rolling(_return_period).max().shift(-_return_period)
    )
    max_pct_period = (max_price_period - stk_data["Adj Close"]) / stk_data["Adj Close"]
    pct_label = max_pct_period.apply(lambda x: 1 if x >= pct_threshold else 0).astype(
        "int8"
    )
    pct_label.name = "label"
    return pct_label


def class_percentage(analysis_data):
    stat = analysis_data.groupby("label").size()
    total = len(analysis_data)
    p = []
    for i in range(num_classes):
        p.append(stat[i] / total if i in stat.index else 0.0)
    return p

In [24]:
def to_adjusted(stk_data):
    """
    Adjusted Open = Open * Adjusted Close / Close
    Adjusted High = High * Adjusted Close / Close
    Adjusted Low = Low * Adjusted Close / Close
    Adjusted volume = Volume / (Adjusted Close / Close)
    """
    ratio_data = stk_data["Adj Close"] / stk_data["Close"]
    adjusted_OHLV = pd.DataFrame(index=stk_data.index)
    adjusted_OHLV["Adj Open"] = ratio_data * stk_data["Open"]
    adjusted_OHLV["Adj High"] = ratio_data * stk_data["High"]
    adjusted_OHLV["Adj Low"] = ratio_data * stk_data["Low"]
    adjusted_OHLV["Adj Close"] = stk_data["Adj Close"]
    adjusted_OHLV["Adj Volume"] = (ratio_data * stk_data["Volume"]).astype("int")
    adjusted_OHLV["Pre Adj Close"] = stk_data["Adj Close"].shift(1)
    adjusted_OHLV["Pre Adj Volume"] = adjusted_OHLV["Adj Volume"].shift(1)
    adjusted_OHLV = adjusted_OHLV.dropna()
    adjusted_OHLV["Pre Adj Volume"] = adjusted_OHLV["Pre Adj Volume"].astype("int")
    adjusted_OHLV["Adj Open pct"] = (
        adjusted_OHLV["Adj Open"] - adjusted_OHLV["Pre Adj Close"]
    ) / adjusted_OHLV["Pre Adj Close"]
    adjusted_OHLV["Adj High pct"] = (
        adjusted_OHLV["Adj High"] - adjusted_OHLV["Pre Adj Close"]
    ) / adjusted_OHLV["Pre Adj Close"]
    adjusted_OHLV["Adj Low pct"] = (
        adjusted_OHLV["Adj Low"] - adjusted_OHLV["Pre Adj Close"]
    ) / adjusted_OHLV["Pre Adj Close"]
    adjusted_OHLV["Adj Volume pct"] = (
        adjusted_OHLV["Adj Volume"] - adjusted_OHLV["Pre Adj Volume"]
    ) / adjusted_OHLV["Pre Adj Volume"]

    return adjusted_OHLV

In [ ]:
import pandas_ta as ta

# help(ta.kvo)
# help(ta.adosc)
stk_data = ticks_data[0]
print(stk_data)
adjusted_OHLV = to_adjusted(stk_data)
print(adjusted_OHLV.head(20))
print(adjusted_OHLV.ta.stoch(high="Adj High", low="Adj Low", close="Adj Close", k=10))
print(stk_data.ta.stoch(k=10))

print(
    adjusted_OHLV.ta.adosc(
        high="Adj High", low="Adj Low", close="Adj Close", volume="Adj Volume"
    )
)
print(stk_data.ta.adosc())

In [25]:
def gen_buy_sell_signal(stk_data):
    import pandas_ta as ta

    sma = pd.concat(
        [
            stk_data.ta.sma(close="Adj Close", length=10),
            stk_data.ta.sma(close="Adj Close", length=60),
        ],
        axis=1,
    ).dropna()
    buy_signal = sma["SMA_10"] > sma["SMA_60"]

    buy_sell_signal = stk_data[[]].copy()
    buy_sell_signal["Signal"] = (buy_signal).astype("int")

    return buy_sell_signal


def gen_analysis_data(stk_data, _return_period):
    import pandas_ta as ta

    adjusted_OHLV = to_adjusted(stk_data)
    data = pd.concat(
        [
            adjusted_OHLV.ta.adosc(
                high="Adj High", low="Adj Low", close="Adj Close", volume="Adj Volume"
            ),
            adjusted_OHLV.ta.kvo(
                high="Adj High", low="Adj Low", close="Adj Close", volume="Adj Volume"
            ),
            stk_data.ta.rsi(close="Adj Close", length=10) / 100,
            stk_data.ta.rsi(close="Adj Close", length=30) / 100,
            stk_data.ta.rsi(close="Adj Close", length=200) / 100,
            stk_data.ta.stoch(k=10) / 100,
            stk_data.ta.stoch(k=30) / 100,
            stk_data.ta.stoch(k=200) / 100,
            adjusted_OHLV.loc[
                :, ["Adj Open pct", "Adj High pct", "Adj Low pct", "Adj Volume pct"]
            ],
            gen_buy_sell_signal(stk_data),
        ],
        axis=1,
    )

    data = pd.concat(
        [data.astype("float32"), gen_pct_label(stk_data, _return_period)],
        axis=1,
    ).dropna()
    return data


def prepare_analysis_data(_return_period, verbose=False):
    from tqdm import tqdm

    ticks_dataset = []
    ignore_ticks_data_count = 0
    for i, tick_data in enumerate(tqdm(ticks_data)):
        analysis_data = gen_analysis_data(tick_data, _return_period)
        classes_percentage = class_percentage(analysis_data)
        if 0 in classes_percentage:
            if verbose:
                print(
                    f"Some classes don't have any data  : {stk_symbols[i]}, {classes_percentage}"
                )
            ignore_ticks_data_count += 1
        elif any(p < class_percentage_threshold for p in classes_percentage):
            if verbose:
                print(
                    f"Some classes are too small  : {stk_symbols[i]}, {classes_percentage}"
                )
            ignore_ticks_data_count += 1
        else:
            ticks_dataset.append(analysis_data)
    if ignore_ticks_data_count > 0:
        print(
            f"There are {ignore_ticks_data_count} stocks in total, some classes have no data or are too small"
        )
    return ticks_dataset

In [13]:
stk_data = ticks_data[149]
d = stk_data.describe()
print(d)
# print(type(d).__name__)
# print((d.loc["count"] == 0).any())
print(is_contained(d.loc[:, ["Open", "High", "Low", "Close", "Adj Close"]], 0))

# xx = gen_analysis_data(stk_data, return_period)
# print(xx.sort_values("Adj Volume pct", ascending=False))
# xx.describe()
# sd = pd.DataFrame()
# # sd.drop()
for i, stk_data in enumerate(ticks_data):
    if is_contained(stk_data.loc[:, ["Open", "High", "Low", "Close", "Adj Close"]], 0):
        print(f"{i},{stk_symbols[i]}")
    if is_contained(stk_data.describe().loc["std"], 0):
        print(f"{i},{stk_symbols[i]}")
    elif (stk_data["Volume"] == 0).sum() > 0:
        print(f"{i},{stk_symbols[i]}")
        xx = stk_data.drop(stk_data[stk_data["Volume"] == 0].index)
        print(xx.describe())
# if (stk_data["Volume"] == 0).sum() > 0:
#     print(stk_data.describe())
#     print(f"{i},{stk_symbols[i]}")
#     xx = stk_data.drop(stk_data[stk_data["Volume"] == 0].index)
#     print(xx.describe())

          Open     High      Low    Close  Adj Close   Volume
count 1895.000 1895.000 1895.000 1895.000   1895.000 1895.000
mean   138.768  139.178  138.372  138.781    138.781    0.000
std     13.660   13.624   13.688   13.655     13.655    0.000
min    115.170  116.750  111.300  114.580    114.580    0.000
25%    128.855  129.240  128.485  128.885    128.885    0.000
50%    132.970  133.600  132.600  133.020    133.020    0.000
75%    149.030  149.495  148.645  148.845    148.845    0.000
max    171.630  171.900  171.410  171.650    171.650    0.000
False
39,AMD
          Open     High      Low    Close  Adj Close        Volume
count 2515.000 2515.000 2515.000 2515.000   2515.000      2515.000
mean    43.123   44.006   42.221   43.131     43.131  54986486.600
std     42.266   43.115   41.380   42.263     42.263  37500234.414
min      1.620    1.690    1.610    1.620      1.620   2606600.000
25%      5.115    5.190    5.000    5.110      5.110  29471700.000
50%     23.660   24.230   2

In [26]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import math
from MyPyUtil.util import ivmax


class LSTMDataSet(Dataset):
    def __init__(self, ticks_data_X, ticks_data_Y, _seq_len, balanced, pattern):
        self.ticks_data_X = ticks_data_X
        self.ticks_data_Y = ticks_data_Y
        self.seq_len = _seq_len
        self.balanced = balanced
        self.pattern = pattern
        self.pattern_size = 0 if pattern == None else len(pattern)

        len_array = [len(d) - self.seq_len + 1 for d in ticks_data_X]
        self.idx_boundary = [len_array[0]]

        for i in range(1, len(len_array)):
            self.idx_boundary.append(len_array[i] + self.idx_boundary[i - 1])

        self.build_class_indices()
        if self.balanced and self.pattern != None and len(self.pattern) > 0:
            self.build_pattern_info()

    def build_class_indices(self):
        total_y = pd.concat(
            [t[self.seq_len - 1 :]["label"] for t in self.ticks_data_Y]
        ).reset_index()
        self.class_indices = []
        for i in range(num_classes):
            class_idx_list = total_y.index[total_y["label"] == i].tolist()
            # random.shuffle(class_idx_list)
            self.class_indices.append(class_idx_list)

        self.class_num_of_max_size, self.max_class_size = ivmax(
            [len(class_idx_list) for class_idx_list in self.class_indices]
        )

    def build_pattern_info(self):
        self.inner_class_count_of_pattern = list(np.zeros(num_classes, dtype=int))
        self.inner_offset_of_pattern = list(np.zeros(self.pattern_size, dtype=int))
        for i, c in enumerate(self.pattern):
            self.inner_offset_of_pattern[i] = self.inner_class_count_of_pattern[c]
            self.inner_class_count_of_pattern[c] += 1

    def __len__(self):
        if self.balanced:
            if self.pattern != None and len(self.pattern) > 0:
                return math.ceil(
                    self.max_class_size
                    / self.inner_class_count_of_pattern[self.class_num_of_max_size]
                ) * len(self.pattern)
                # return math.ceil(
                #     self.max_class_size
                #     * (
                #         len(self.pattern)
                #         / self.inner_class_count_of_pattern[self.class_num_of_max_size]
                #     )
                # )
            else:
                return self.max_class_size * num_classes

        return self.idx_boundary[-1]

    def info(self):
        print(f"Dataset size: {self.idx_boundary[-1]}")
        for i in range(num_classes):
            print(
                f"class {i}: {len(self.class_indices[i]) * 100 /self.idx_boundary[-1]:.1f}% {len(self.class_indices[i])}"
            )
        if self.balanced:
            new_size = self.__len__()
            print(f"\nNew dataset size after balancing classes: {new_size}")
            if self.pattern != None and len(self.pattern) > 0:
                for i in range(num_classes):
                    ratio = self.inner_class_count_of_pattern[i] / len(self.pattern)
                    print(f"class {i}: {100 * ratio:.1f}% {new_size * ratio:g}")
            # size_ary = list(np.zeros(num_classes))
            # size_ary[self.class_num_of_max_size] = self.max_class_size
            # tmp_total = self.max_class_size
            # for i in range(num_classes):
            #     if i == self.class_num_of_max_size:
            #         print(
            #             f"class {i}: {100 * size_ary[i]/new_size:.1f}% {size_ary[i]}"
            #         )
            #         continue

            #     if i == num_classes - 1:
            #         size_ary[i] = new_size - tmp_total
            #     else:
            #         if (new_size % self.pattern_size) == 0:
            #             size_ary[i] = (
            #                 new_size / self.pattern_size
            #             ) * self.inner_class_count_of_pattern[i]
            #         else:
            #             size_ary[i] = (
            #                 math.ceil(new_size / self.pattern_size)
            #                 if i < self.class_num_of_max_size
            #                 else math.floor(new_size / self.pattern_size)
            #             ) * self.inner_class_count_of_pattern[i]
            #         tmp_total += size_ary[i]

            else:
                for i in range(num_classes):
                    print(f"class {i}: {100/num_classes:.1f}% {self.max_class_size}")

    def idx_of_balanced_data_to_original_idx(self, idx_of_balanced_data):
        if self.pattern != None and self.pattern_size > 0:
            pattern_idx = idx_of_balanced_data % self.pattern_size
            selected_class = self.pattern[pattern_idx]
            idx_of_balanced_class = (
                (idx_of_balanced_data // self.pattern_size)
                * self.inner_class_count_of_pattern[selected_class]
            ) + self.inner_offset_of_pattern[pattern_idx]
        else:
            selected_class = idx_of_balanced_data % num_classes
            idx_of_balanced_class = idx_of_balanced_data // num_classes

        offset_balanced_class = idx_of_balanced_class % len(
            self.class_indices[selected_class]
        )
        return self.class_indices[selected_class][offset_balanced_class]

    def __getitem__(self, idx_of_balanced_data):
        idx = (
            self.idx_of_balanced_data_to_original_idx(idx_of_balanced_data)
            if self.balanced
            else idx_of_balanced_data
        )

        # print(f"getitem, idx_of_balanced_data:{idx_of_balanced_data}, idx:{idx}")
        for ticks_data_idx in range(len(self.ticks_data_X)):
            if self.idx_boundary[ticks_data_idx] > idx:
                break
        offset = (
            idx if ticks_data_idx == 0 else idx - self.idx_boundary[ticks_data_idx - 1]
        )
        x = np.array(self.ticks_data_X[ticks_data_idx][offset : offset + self.seq_len])
        y = int(self.ticks_data_Y[ticks_data_idx].iloc[offset + self.seq_len - 1, :])
        return (x, y)

In [29]:
import random
import math
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn_pandas import DataFrameMapper


def prepare_LSTMDataset(_return_period, _seq_len, train_data_pattern=None):
    ticks_dataset = prepare_analysis_data(_return_period)
    ticks_X_train_data = []
    ticks_Y_train_data = []
    ticks_X_test_data = []
    ticks_Y_test_data = []
    ticks_X_dfm = []
    for idx, dataset in enumerate(ticks_dataset):
        # test_size = int(dataset.shape[0] * validation_size)
        train_size = int(dataset.shape[0] * (1 - validation_size))
        # random.seed(42)
        train_data = dataset.iloc[0:train_size]
        test_data = dataset.iloc[train_size - seq_len + 1 :]

        X_train_data = train_data.iloc[:, :-1]
        Y_train_data = train_data.iloc[:, -1:]

        X_test_data = test_data.iloc[:, :-1]
        Y_test_data = test_data.iloc[:, -1:]

        features = [
            ([column], StandardScaler()) for column in X_train_data.columns[:-1].values
        ]
        features.extend(
            [([column], None) for column in X_train_data.columns[-1:].values]
        )
        if len(features) != 17:
            print(f"Wrong data: {stk_symbols[idx]}, {X_train_data.columns.values}")
            break
        # print(idx)
        X_dfm = DataFrameMapper(features, input_df=True, df_out=True)
        X_train_data = X_dfm.fit_transform(X_train_data)
        X_test_data = X_dfm.transform(X_test_data)

        ticks_X_dfm.append(X_dfm)
        ticks_X_train_data.append(X_train_data)
        ticks_Y_train_data.append(Y_train_data)
        ticks_X_test_data.append(X_test_data)
        ticks_Y_test_data.append(Y_test_data)

    train_dataset = LSTMDataSet(
        ticks_X_train_data,
        ticks_Y_train_data,
        _seq_len,
        balanced=True,
        pattern=train_data_pattern,
    )
    test_dataset = LSTMDataSet(
        ticks_X_test_data, ticks_Y_test_data, _seq_len, balanced=False, pattern=None
    )

    print("Training data:")
    train_dataset.info()
    print("\nTest data")
    test_dataset.info()

    return [train_dataset, test_dataset]

In [ ]:
prepare_LSTMDataset(return_period, seq_len)

In [ ]:
t, s = prepare_LSTMDataset(return_period, seq_len, [0, 1, 0])

In [30]:
import random
import math
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn_pandas import DataFrameMapper


def prepare_dataloader(_return_period, _seq_len, train_data_pattern=None):
    data = prepare_LSTMDataset(_return_period, _seq_len, train_data_pattern)

    train_loader = DataLoader(
        data[0],
        batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory,
        pin_memory_device=device_name,
    )
    test_loader = DataLoader(
        data[1],
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        pin_memory_device=device_name,
    )

    return train_loader, test_loader, data[0].ticks_data_X[0].shape[1]

In [31]:
from torch import nn


class StockPCTLabelPredictLSTM(nn.Module):
    def __init__(
        self,
        input_size,
        hidden_size,
        num_layers,
        num_fc_layers,
        activation_type,
    ):
        super().__init__()
        self.setup_model(
            input_size,
            hidden_size,
            num_layers,
            num_fc_layers,
            activation_type,
        )

    def __init__(self, input_size, config):
        super().__init__()
        self.setup_model(
            input_size=input_size,
            hidden_size=config["hidden_size"],
            num_layers=config["num_layers"],
            num_fc_layers=config["num_fc_layers"],
            activation_type=config["activation_type"],
        )

    def setup_model(
        self,
        input_size,
        hidden_size,
        num_layers,
        num_fc_layers,
        activation_type,
    ):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        """
            input_size    : The number of expected features in the input x
            hidden_size   : The number of features in the hidden state h
            num_layers    : Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two LSTMs together to form a stacked LSTM, with the second LSTM taking in outputs of the first LSTM and computing the final results. Default: 1
            bias          : If False, then the layer does not use bias weights b_ih and b_hh. Default: True
            batch_first   : If True, then the input and output tensors are provided as (batch, seq, feature) instead of (seq, batch, feature). Note that this does not apply to hidden or cell states. See the Inputs/Outputs sections below for details. Default: False
            dropout       : If non-zero, introduces a Dropout layer on the outputs of each LSTM layer except the last layer, with dropout probability equal to dropout. Default: 0
            bidirectional : If True, becomes a bidirectional LSTM. Default: False
            proj_size     : If > 0, will use LSTM with projections of corresponding size. Default: 0
        """
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        layers = []
        in_features = self.hidden_size
        for i in range(1, num_fc_layers):
            out_features = int(in_features / 2)
            if out_features <= num_classes:
                break
            layers.append(nn.Linear(in_features, out_features))
            (
                layers.append(nn.ReLU() if activation_type == 1 else nn.Sigmoid())
                if activation_type == 2
                else nn.Tanh()
            )
            in_features = out_features

        layers.append(nn.Linear(in_features, num_classes))
        self.fc = nn.Sequential(*layers)
        self.fc.apply(self.init_weights)

    def init_weights(self, m):
        if isinstance(m, nn.Linear):
            initrange = 0.5
            nn.init.uniform_(m.weight, -initrange, initrange)
            nn.init.zeros_(m.bias)

    def forward(self, x):
        h_0 = torch.zeros(self.num_layers, x.shape[0], self.hidden_size).to(device)
        c_0 = torch.zeros(self.num_layers, x.shape[0], self.hidden_size).to(device)
        out, (h_out, _) = self.rnn(x, (h_0, c_0))

        fc_input = h_out[-1].view(-1, self.hidden_size)
        return self.fc(fc_input)


def save_model(model, hyper_parameters, file_path, epoch_num=None):
    state = {
        "epoch_num": epoch_num,
        "time": str(datetime.now),
        "model_state": model.state_dict(),
        "input_size": model.input_size,
        "hyper_parameters": hyper_parameters,
    }
    torch.save(state, file_path)


def load_model(file_path):
    data_dict = torch.load(file_path)
    hyper_parameters = data_dict["hyper_parameters"]
    model = StockPCTLabelPredictLSTM(
        input_size=data_dict["input_size"],
        hidden_size=int(hyper_parameters["hidden_size"]),
        num_layers=int(hyper_parameters["num_layers"]),
        num_fc_layers=int(hyper_parameters["num_fc_layers"]),
        activation_type=int(hyper_parameters["activation_type"]),
    )
    model.load_state_dict(data_dict["model_state"])
    return model, hyper_parameters

In [32]:
import torch.nn as nn

METRICS_LABEL_NDX = 0  # ground_truth
METRICS_PBTY_NDX = 1  # Probability of predicition
METRICS_PRED_NDX = 2  # class(label) of predicition
METRICS_LOSS_NDX = 3
METRICS_SIZE = 4
softmax = nn.Softmax(dim=1)
totalTrainingSamples_count = 0

In [33]:
from collections import namedtuple
from torch.utils.tensorboard import SummaryWriter


def logMetrics(
    epoch_ndx,
    mode_str,
    metrics_t,
    classificationThreshold=0.5,
    config=None,
    log_hparam=False,
):
    log.info(
        "E{} {}".format(
            epoch_ndx,
            task_name,
        )
    )
    F1_rec = namedtuple(
        "f1_rec",
        "target_class pos_correct neg_correct pos_count neg_count pos_loss neg_loss precision recall F1",
    )
    F1_metrics = []
    for target_class in reversed(range(num_classes)):
        posLabel_mask = metrics_t[METRICS_LABEL_NDX] == target_class
        pos_count = posLabel_mask.sum()
        negLabel_mask = metrics_t[METRICS_LABEL_NDX] != target_class
        neg_count = negLabel_mask.sum()

        posPred_mask = metrics_t[METRICS_PRED_NDX] == target_class
        threshold_mask = metrics_t[METRICS_PBTY_NDX] > classificationThreshold
        # TP, truePos_count
        TP = pos_correct = int((posLabel_mask & posPred_mask & threshold_mask).sum())

        negPred_mask = metrics_t[METRICS_PRED_NDX] != target_class
        # TN, trueNeg_count
        TN = neg_correct = int((negLabel_mask & negPred_mask).sum())

        # FP, falsePos_count
        FP = neg_count - neg_correct
        # FN, falseNeg_count
        FN = pos_count - pos_correct

        # precision = TP / (TP + FP)
        precision = 0.0 if (TP + FP) == 0 else TP / np.float32(TP + FP)
        # recall = TP / (TP + FN)
        recall = 0.0 if (TP + FN) == 0 else TP / np.float32(TP + FN)
        # F1 = 2 * precision * recall / (precision + recall)
        F1 = (
            0.0
            if (precision + recall) == 0.0
            else (2 * precision * recall) / np.float32(precision + recall)
        )
        F1_metrics.append(
            F1_rec(
                target_class,
                pos_correct,
                neg_correct,
                pos_count,
                neg_count,
                metrics_t[METRICS_LOSS_NDX, posLabel_mask].mean(),
                metrics_t[METRICS_LOSS_NDX, negLabel_mask].mean(),
                precision,
                recall,
                F1,
            )
        )

        if num_classes == 2:
            break

    metrics_dict = {}
    metrics_dict[" e_loss/all"] = metrics_t[METRICS_LOSS_NDX].mean()
    log.info(
        ("E{} {:8} { e_loss/all:.4f} loss").format(
            epoch_ndx,
            mode_str,
            **metrics_dict,
        )
    )

    for target_class, rec in enumerate(F1_metrics):
        target_class_str = f"class {rec.target_class}" if num_classes > 2 else ""
        metrics_dict[f"{target_class_str} e_loss/pos"] = rec.pos_loss
        metrics_dict[f"{target_class_str} e_loss/neg"] = rec.neg_loss

        metrics_dict[f"{target_class_str} correct/all"] = (
            (rec.pos_correct + rec.neg_correct) / metrics_t.shape[1] * 100
        )
        metrics_dict[f"{target_class_str} correct/neg"] = (
            (rec.neg_correct) / rec.neg_count * 100
        )
        metrics_dict[f"{target_class_str} correct/pos"] = (
            (rec.pos_correct) / rec.pos_count * 100
        )
        metrics_dict[f"{target_class_str} pr/precision"] = rec.precision
        metrics_dict[f"{target_class_str} pr/recall"] = rec.recall
        metrics_dict[f"{target_class_str} pr/f1_score"] = rec.F1

        log.info(
            (
                "E{} {:8} {} {"
                + f"{target_class_str}"
                + " correct/all:-5.1f}% correct, "
                + "{"
                + f"{target_class_str}"
                + " pr/precision:.4f} precision, "
                + "{"
                + f"{target_class_str}"
                + " pr/recall:.4f} recall, "
                + "{"
                + f"{target_class_str}"
                + " pr/f1_score:.4f} f1 score"
            ).format(epoch_ndx, mode_str, target_class_str, **metrics_dict)
        )
        log.info(
            (
                "E{} {:8} {} {"
                + f"{target_class_str}"
                + " e_loss/neg:.4f} loss, "
                + "{"
                + f"{target_class_str}"
                + " correct/neg:-5.1f}% correct ({neg_correct:} of {neg_count:})"
            ).format(
                epoch_ndx,
                mode_str + "_neg",
                target_class_str,
                neg_correct=rec.neg_correct,
                neg_count=rec.neg_count,
                **metrics_dict,
            )
        )
        log.info(
            (
                "E{} {:8} {} {"
                + f"{target_class_str}"
                + " e_loss/pos:.4f} loss, "
                + "{"
                + f"{target_class_str}"
                + " correct/pos:-5.1f}% correct ({pos_correct:} of {pos_count:})"
            ).format(
                epoch_ndx,
                mode_str + "_pos",
                target_class_str,
                pos_correct=rec.pos_correct,
                pos_count=rec.pos_count,
                **metrics_dict,
            )
        )

    writer = SummaryWriter(log_dir=log_dir + f"/{mode_str}_cls")
    for key, value in metrics_dict.items():
        writer.add_scalar(key, value, totalTrainingSamples_count)

    writer.add_pr_curve(
        "pr",
        metrics_t[METRICS_LABEL_NDX],
        metrics_t[METRICS_PRED_NDX],
        totalTrainingSamples_count,
    )

    bins = [x / 50.0 for x in range(51)]
    negHist_mask = negLabel_mask & (metrics_t[METRICS_PBTY_NDX] > 0.01)
    posHist_mask = posLabel_mask & (metrics_t[METRICS_PBTY_NDX] < 0.99)
    if negHist_mask.any():
        writer.add_histogram(
            "is_neg",
            metrics_t[METRICS_PBTY_NDX, negHist_mask],
            totalTrainingSamples_count,
            bins=bins,
        )
    if posHist_mask.any():
        writer.add_histogram(
            "is_pos",
            metrics_t[METRICS_PBTY_NDX, posHist_mask],
            totalTrainingSamples_count,
            bins=bins,
        )

    if log_hparam:
        hparam = config.copy()
        hparam["0:trn,1:val"] = 0 if mode_str == "trn" else 1
        writer.add_hparams(
            hparam,
            {
                "loss": metrics_t[METRICS_LOSS_NDX].mean(),
                "F1": F1_metrics[-1].F1,
            },
        )

    writer.close()

    return float(metrics_dict[" e_loss/all"]), F1_metrics

2024-02-08 09:57:46.712561: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-08 09:57:46.713891: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-08 09:57:46.734207: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-08 09:57:47.125065: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [34]:
import sys


def computeBatchLoss(model, loss_fn, x, y, metrics_g, batch_idx):
    x_g = x.to(device)
    y_g = y.to(device)
    outputs = model(x_g)
    if outputs.isnan().sum() > 0:
        return sys.float_info.max

    loss_g = loss_fn(outputs, y_g)
    probability_g, predition_g = torch.max(softmax(outputs), dim=1)

    start_ndx = batch_idx * batch_size
    end_ndx = start_ndx + y.size(0)

    metrics_g[METRICS_LABEL_NDX, start_ndx:end_ndx] = y_g
    metrics_g[METRICS_PBTY_NDX, start_ndx:end_ndx] = probability_g
    metrics_g[METRICS_PRED_NDX, start_ndx:end_ndx] = predition_g
    metrics_g[METRICS_LOSS_NDX, start_ndx:end_ndx] = loss_g
    loss = loss_g.mean()
    return loss

In [37]:
from MyPyUtil.util import enumerateWithEstimate
import torch
from tqdm import tqdm


def doTraining(model, optimizer, loss_fn, epoch_ndx, train_dl):
    global totalTrainingSamples_count
    model.train()
    trnMetrics_g = torch.zeros(
        METRICS_SIZE,
        len(train_dl.dataset),
        device=device,
    )

    batch_iter = enumerateWithEstimate(
        train_dl,
        "E{} Training".format(epoch_ndx),
        start_ndx=train_dl.num_workers,
    )
    for batch_ndx, (x, y) in batch_iter:
        # for batch_ndx, (x, y) in enumerate(tqdm(train_dl)):
        optimizer.zero_grad()

        loss = computeBatchLoss(
            model,
            loss_fn,
            x,
            y,
            trnMetrics_g,
            batch_ndx,
        )

        if loss == sys.float_info.max:
            print(f"forward error: {batch_ndx}")
        else:
            loss.backward()
            optimizer.step()

    totalTrainingSamples_count += len(train_dl.dataset)
    return trnMetrics_g.to("cpu")

In [38]:
def doValidation(model, loss_fn, epoch_ndx, val_dl):
    from sklearn.metrics import f1_score

    with torch.no_grad():
        model.eval()
        valMetrics_g = torch.zeros(
            METRICS_SIZE,
            len(val_dl.dataset),
            device=device,
        )

        batch_iter = enumerateWithEstimate(
            val_dl,
            "E{} Validation ".format(epoch_ndx),
            start_ndx=val_dl.num_workers,
        )
        for batch_ndx, (x, y) in batch_iter:
            # for batch_ndx, (x, y) in enumerate(tqdm(val_dl)):
            loss = computeBatchLoss(model, loss_fn, x, y, valMetrics_g, batch_ndx)
            if loss == sys.float_info.max:
                print(f"forward error: {batch_ndx}")

    return valMetrics_g.to("cpu")

In [39]:
def train_LSTM(config):
    global totalTrainingSamples_count
    best_f1 = 0

    lr = config["lr"]
    momentum = config["momentum"]
    optim_type = config["optim_type"]
    totalTrainingSamples_count = 0

    id_str = "_".join(str(v) if v < 1 else f"{v:g}" for v in config.values())
    # print(id_str)
    model_name = f"{log_dir}/{id_str}.pt"

    train_loader, test_loader, features_size = prepare_dataloader(
        config["return_period"], config["seq_len"], train_data_pattern=[0, 0, 1]
    )

    model = StockPCTLabelPredictLSTM(input_size=features_size, config=config)
    model = model.to(device)

    optimizer = (
        torch.optim.Adam(model.parameters(), lr=lr)
        if optim_type == 1
        else torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    )
    loss_fn = torch.nn.CrossEntropyLoss(reduction="none")

    for epoch_ndx in range(epoch_num):
        trnMetrics_t = doTraining(model, optimizer, loss_fn, epoch_ndx, train_loader)
        loss, _ = logMetrics(
            epoch_ndx,
            "trn",
            trnMetrics_t,
            classificationThreshold,
            config,
            (epoch_ndx == epoch_num - 1),
        )

        valMetrics_t = doValidation(model, loss_fn, epoch_ndx, test_loader)
        _, F1_metrics = logMetrics(
            epoch_ndx,
            "val",
            valMetrics_t,
            classificationThreshold,
            config,
            (epoch_ndx == epoch_num - 1),
        )
        if F1_metrics[0].F1 > best_f1:
            best_f1 = F1_metrics[0].F1
            save_model(model, config, model_name)
            print(f"current loss: {loss}")

In [40]:
time_str = datetime.now().strftime("%Y-%m-%d_%H.%M.%S")
log_dir = f"{log_dir_base}/{time_str}"
config = {
    "return_period": return_period,
    "seq_len": seq_len,
    "lr": 0.1,
    "momentum": 0.11646759543664197,
    "optim_type": 2,  # 1: Adam, 2: SGD  => Adam bad result
    "weight decay": 0.00001,
    "num_layers": 4,
    "hidden_size": 256,
    "num_fc_layers": 1,
    "activation_type": 2,  # Sigmoid
}
epoch_num = 100
# os.mkdir(log_dir)
print(log_dir)
start = datetime.now()
torch.seed = 42
random.seed(42)
np.random.seed(42)
train_LSTM(config)
print(f"Elapsed time:{datetime.now() - start}")

/mnt/AIWorkSpace/work/fin-ml/runs/StockPCTLabelPredictLSTMV2/2024-02-08_09.59.43


100%|██████████| 494/494 [00:06<00:00, 78.10it/s]
2024-02-08 10:00:02,076 INFO     pid:26301 MyPyUtil.util:146:enumerateWithEstimate E0 Training ----/37103, starting
2024-02-08 10:00:02,158 INFO     pid:26301 MyPyUtil.util:166:enumerateWithEstimate E0 Training    4/37103, done at 2024-02-08 10:10:02, 0:10:00


Training data:
Dataset size: 897349
class 0: 88.2% 791528
class 1: 11.8% 105821

New dataset size after balancing classes: 1187292
class 0: 66.7% 791528
class 1: 33.3% 395764

Test data
Dataset size: 224838
class 0: 84.8% 190583
class 1: 15.2% 34255


2024-02-08 10:00:02,262 INFO     pid:26301 MyPyUtil.util:166:enumerateWithEstimate E0 Training   32/37103, done at 2024-02-08 10:03:29, 0:03:27
2024-02-08 10:00:03,062 INFO     pid:26301 MyPyUtil.util:166:enumerateWithEstimate E0 Training  256/37103, done at 2024-02-08 10:02:24, 0:02:22
2024-02-08 10:00:10,560 INFO     pid:26301 MyPyUtil.util:166:enumerateWithEstimate E0 Training 2048/37103, done at 2024-02-08 10:02:35, 0:02:33
2024-02-08 10:01:05,223 INFO     pid:26301 MyPyUtil.util:166:enumerateWithEstimate E0 Training 16384/37103, done at 2024-02-08 10:02:25, 0:02:22
2024-02-08 10:02:24,545 INFO     pid:26301 MyPyUtil.util:181:enumerateWithEstimate E0 Training 37103/37103, done at 2024-02-08 10:02:24
2024-02-08 10:02:24,550 INFO     pid:26301 __main__:013:logMetrics E0 StockPCTLabelPredictLSTMV2
2024-02-08 10:02:24,560 INFO     pid:26301 __main__:074:logMetrics E0 trn      0.5761 loss
2024-02-08 10:02:24,561 INFO     pid:26301 __main__:100:logMetrics E0 trn        71.4% correct, 0.6

current loss: 0.5760806202888489


...

current loss: 0.5633395910263062


2024-02-08 10:08:15,121 INFO     pid:26301 MyPyUtil.util:166:enumerateWithEstimate E3 Training  256/37103, done at 2024-02-08 10:10:26, 0:02:12
2024-02-08 10:08:21,444 INFO     pid:26301 MyPyUtil.util:166:enumerateWithEstimate E3 Training 2048/37103, done at 2024-02-08 10:10:25, 0:02:11
2024-02-08 10:09:12,861 INFO     pid:26301 MyPyUtil.util:166:enumerateWithEstimate E3 Training 16384/37103, done at 2024-02-08 10:10:27, 0:02:12
2024-02-08 10:10:33,305 INFO     pid:26301 MyPyUtil.util:181:enumerateWithEstimate E3 Training 37103/37103, done at 2024-02-08 10:10:33
2024-02-08 10:10:33,666 INFO     pid:26301 __main__:013:logMetrics E3 StockPCTLabelPredictLSTMV2
2024-02-08 10:10:33,696 INFO     pid:26301 __main__:074:logMetrics E3 trn      0.5599 loss
2024-02-08 10:10:33,697 INFO     pid:26301 __main__:100:logMetrics E3 trn        72.2% correct, 0.6617 precision, 0.3391 recall, 0.4484 f1 score
2024-02-08 10:10:33,697 INFO     pid:26301 __main__:116:logMetrics E3 trn_neg   0.3599 loss,  91.3

current loss: 0.5599337816238403


2024-02-08 10:10:56,053 INFO     pid:26301 MyPyUtil.util:166:enumerateWithEstimate E4 Training  256/37103, done at 2024-02-08 10:13:06, 0:02:11
2024-02-08 10:11:02,354 INFO     pid:26301 MyPyUtil.util:166:enumerateWithEstimate E4 Training 2048/37103, done at 2024-02-08 10:13:05, 0:02:10
2024-02-08 10:11:58,938 INFO     pid:26301 MyPyUtil.util:166:enumerateWithEstimate E4 Training 16384/37103, done at 2024-02-08 10:13:19, 0:02:24
2024-02-08 10:13:14,901 INFO     pid:26301 MyPyUtil.util:181:enumerateWithEstimate E4 Training 37103/37103, done at 2024-02-08 10:13:14
2024-02-08 10:13:15,172 INFO     pid:26301 __main__:013:logMetrics E4 StockPCTLabelPredictLSTMV2
2024-02-08 10:13:15,207 INFO     pid:26301 __main__:074:logMetrics E4 trn      0.5574 loss
2024-02-08 10:13:15,207 INFO     pid:26301 __main__:100:logMetrics E4 trn        72.4% correct, 0.6668 precision, 0.3443 recall, 0.4541 f1 score
2024-02-08 10:13:15,207 INFO     pid:26301 __main__:116:logMetrics E4 trn_neg   0.3581 loss,  91.4

: 

In [18]:
def ray_train_task(config, data):
    global totalTrainingSamples_count
    global log_dir

    best_f1 = 0

    lr = config["lr"]
    momentum = config["momentum"]
    optim_type = config["optim_type"]
    totalTrainingSamples_count = 0

    id_str = "_".join(str(v) if v < 1 else f"{v:g}" for v in config.values())
    # print(id_str)
    log_dir = f"{log_dir_base}/{time_str}/{id_str}"
    os.mkdir(log_dir)

    model_name = f"{log_dir}/{id_str}.pt"

    train_loader = DataLoader(
        data[0],
        batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory,
        pin_memory_device=device_name,
    )
    test_loader = DataLoader(
        data[1],
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        pin_memory_device=device_name,
    )

    features_size = data[0].ticks_data_X[0].shape[1]

    model = StockPCTLabelPredictLSTM(input_size=features_size, config=config)
    model = model.to(device)

    optimizer = (
        torch.optim.Adam(model.parameters(), lr=lr)
        if optim_type == 1
        else torch.optim.SGD(
            model.parameters(),
            lr=lr,
            momentum=momentum,
            weight_decay=config["weight decay"],
        )
    )
    loss_fn = torch.nn.CrossEntropyLoss(reduction="none")

    for epoch_ndx in range(epoch_num):
        trnMetrics_t = doTraining(model, optimizer, loss_fn, epoch_ndx, train_loader)
        loss, _ = logMetrics(
            epoch_ndx,
            "trn",
            trnMetrics_t,
            classificationThreshold,
            config,
            (epoch_ndx == epoch_num - 1),
        )

        valMetrics_t = doValidation(model, loss_fn, epoch_ndx, test_loader)
        _, F1_metrics = logMetrics(
            epoch_ndx,
            "val",
            valMetrics_t,
            classificationThreshold,
            config,
            (epoch_ndx == epoch_num - 1),
        )
        if F1_metrics[0].F1 > best_f1:
            best_f1 = F1_metrics[0].F1
            save_model(model, config, model_name)

        train.report(
            {
                "loss": loss,
                "f1_score": F1_metrics[0].F1,
                "precision": F1_metrics[0].precision,
                "recall": F1_metrics[0].recall,
            }
        )

In [19]:
search_space = {
    # "return_period": tune.grid_search([5]),  # [2,3,5,10]
    # "seq_len": tune.grid_search([3]),  # 10]),
    "lr": tune.grid_search([0.01, 0.1]),  # , 0.01, 0.1, 0.08, 0.12]
    "momentum": tune.grid_search([0.14647]),  # tune.uniform(0.1, 0.9),
    "optim_type": tune.grid_search([2]),  # 1: Adam, 2: SGD  => Adam bad result
    "weight decay": tune.grid_search([0.00001]),  # best value
    "num_layers": tune.grid_search([4]),  # [1, 2, 4, 8] best value = 3 or 4
    "hidden_size": tune.grid_search([256, 128]),  # [8, 16, 32, 64, 128]
    "num_fc_layers": tune.grid_search([2, 1]),  # 1, 2, 3]),
    "activation_type": tune.grid_search(
        [2]
    ),  # 1: ReLU(),  2: Sigmoid(),  3: Tanh()  => meaningless num_fc_layers == 1
}

turning_parameters = []
total_configs = 1
for k, v in search_space.items():
    if (
        type(v).__name__ == "dict"
        and list(v.keys())[0] == "grid_search"
        and len(list(v.values())[0]) > 1
    ):
        turning_parameters.append(k)
        total_configs *= len(list(v.values())[0])
print(turning_parameters)
print(f"Total count of configs = {total_configs}")

['lr', 'hidden_size', 'num_fc_layers']
Total count of configs = 8


In [20]:
import warnings


warnings.filterwarnings("ignore", category=Warning)

time_str = datetime.now().strftime("%Y-%m-%d_%H.%M.%S")
log_dir = f"{log_dir_base}/{time_str}"
os.mkdir(log_dir)

data = prepare_LSTMDataset(
    return_period, seq_len, train_data_pattern=[0, 1, 0]
)  # [0, 1, 0, 1, 0])
# analysis = tune.run(
#     train_LSTM,
#     config=search_space,
#     resources_per_trial={"cpu": 0.1, "gpu": 0.1},
#     metric="f1_score",
#     mode="max",
# )
tuner = tune.Tuner(
    tune.with_resources(
        tune.with_parameters(ray_train_task, data=data),
        resources={"cpu": 0.33, "gpu": 0.33},
    ),
    tune_config=tune.TuneConfig(
        metric="f1_score",
        mode="max",
    ),
    param_space=search_space,
)
results = tuner.fit()

2024-02-07 21:39:57,505	ERROR tune_controller.py:1374 -- Trial task failed for trial ray_train_task_7c902_00000
Traceback (most recent call last):
  File "/home/skchen/venv/ml4t/lib/python3.8/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/home/skchen/venv/ml4t/lib/python3.8/site-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/skchen/venv/ml4t/lib/python3.8/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/home/skchen/venv/ml4t/lib/python3.8/site-packages/ray/_private/worker.py", line 2624, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(AttributeError): ray::ImplicitFunc.train() (pid=23541, ip=192.168.0.125, actor_id=61078f874fa2d3d927b45e8301000000, repr=ray_train_task)
  File "/home/skchen/venv/ml4t/lib/python3.8/site-packages/ray/tune/trainable/traina